In [282]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
from sklearn.cluster import KMeans

In [264]:
#shamelessly borrowed from https://github.com/r0f1/dev_to_posts/blob/master/fake_data/utils.py
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0
#st.frechet_r st.frechet_l,
    # Distributions to check
    DISTRIBUTIONS = [        
        st.alpha,st.anglit,st.arcsine,st.beta,st.betaprime,st.bradford,st.burr,st.cauchy,st.chi,st.chi2,st.cosine,
        st.dgamma,st.dweibull,st.erlang,st.expon,st.exponnorm,st.exponweib,st.exponpow,st.f,st.fatiguelife,st.fisk,
        st.foldcauchy,st.foldnorm,st.genlogistic,st.genpareto,st.gennorm,st.genexpon,
        st.genextreme,st.gausshyper,st.gamma,st.gengamma,st.genhalflogistic,st.gilbrat,st.gompertz,st.gumbel_r,
        st.gumbel_l,st.halfcauchy,st.halflogistic,st.halfnorm,st.halfgennorm,st.hypsecant,st.invgamma,st.invgauss,
        st.invweibull,st.johnsonsb,st.johnsonsu,st.ksone,st.kstwobign,st.laplace,st.levy,st.levy_l,st.levy_stable,
        st.logistic,st.loggamma,st.loglaplace,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,st.ncf,
        st.nct,st.norm,st.pareto,st.pearson3,st.powerlaw,st.powerlognorm,st.powernorm,st.rdist,st.reciprocal,
        st.rayleigh,st.rice,st.recipinvgauss,st.semicircular,st.t,st.triang,st.truncexpon,st.truncnorm,st.tukeylambda,
        st.uniform,st.vonmises,st.vonmises_line,st.wald,st.weibull_min,st.weibull_max,st.wrapcauchy
    ]

    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for idx, distribution in enumerate(DISTRIBUTIONS):

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass
        
    return (best_distribution.name, best_params)

def make_pdf(dist, params, size=10000):
    """Generate distributions's Propbability Distribution Function """

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get sane start and end points of distribution
    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf


def plot_result(df, continuous, best_distributions):
    for c, (best_fit_name, best_fit_params) in zip(continuous, best_distributions):
        best_dist = getattr(st, best_fit_name)
        pdf = make_pdf(best_dist, best_fit_params)
        _ = plt.figure(figsize=(12,8))
        ax = pdf.plot(lw=2, label='PDF', legend=True)
        _ = df[c].plot(kind='hist', bins=50, normed=True, alpha=0.5, label='Data', legend=True, ax=ax)
        param_names = (best_dist.shapes + ', loc, scale').split(', ') if best_dist.shapes else ['loc', 'scale']
        param_str = ', '.join([f'{k}={v:0.2f}' for k,v in zip(param_names, best_fit_params)])
        dist_str = f'{best_fit_name}({param_str})'
        _ = ax.set_title(c+ " " + dist_str)
        _ = ax.set_ylabel('Frequency')
        plt.show();

In [354]:
df = pd.read_csv('data/2018_Central_Park_Squirrel_Census_-_Squirrel_Data.csv'
)
quasi_ids = [
    'Hectare', 'Date', 'Age', 'Primary Fur Color',
    'Highlight Fur Color', 'Location'
]


In [284]:
#helpers
all_fields = list(df)

fields_list = list(df)
del fields_list[0]

# .remove('id')


In [285]:
# fill in nulls: for numeric use the mean, for categorical use unknown label
# otherwise grab the mode

for c in list(df[quasi_ids]):
    if df[c].dtypes == np.int64:
      df[c]=df[c].fillna((df[c].mean()))
    elif df[c].dtypes == object:
      df[c]=df[c].fillna((df[c].mode()))

#do some silly truncation to reduce the grain

    if df[c].nunique() / df[c].count() >= .005:
      while df[c].nunique() / df[c].count() >= .005 and df[c].map(lambda x: len(str(x))).min() > 1:
        df[c] = df[c].str[:-1]


In [297]:
quasi_ids_cat = list()
quasi_ids_wo_cat = list()

for c in list(df[quasi_ids]):
    # encode as numeric for binning
    if df[c].dtypes != np.int64:
        df[c+'_cat'] = df[c].astype('category')
        df[c+'_cat'] = df[c+'_cat'].cat.codes
        df[c+'_cat'] = pd.to_numeric(df[c+'_cat'],errors = 'coerce')
        quasi_ids_cat.append(c+'_cat')
    else:
        quasi_ids_wo_cat.append(c)
quasi_ids_w_wo_cat = quasi_ids_cat+quasi_ids_wo_cat

df_cat_mapping = df[quasi_ids_cat+quasi_ids].copy().drop_duplicates()

In [292]:
#get the rows that are still problematic
df_aggs = df.groupby(quasi_ids_w_wo_cat).size().reset_index(name="count")
df_slim_cats =  df_aggs[df_aggs['count']<=3] #guessing on this threshold

# for our slim qid cats, we'll employ kmeans to increase bucket size and then
# agg the identifiers together

# we should be more intelligent about cluster count, but I'm in a rush 
kmeans = KMeans(5)

kmeans.fit(df_slim_cats)
identified_clusters = kmeans.fit_predict(df_slim_cats)

data_with_clusters = df_slim_cats.copy()
data_with_clusters['Clusters'] = identified_clusters 


In [293]:
#grab min qid per cluster
#mode would be ideal, but again, low on time and haven't figured out how to make .transform() accept mode
#taking mins here to make sure we get "real" values for things like hectare and date

inner_join_df = pd.merge(df, data_with_clusters, on=quasi_ids_w_wo_cat, how='inner').drop(columns=['count'])
# df
# inner_join_df
for c in list(inner_join_df[quasi_ids_w_wo_cat]):
  inner_join_df[c] = inner_join_df.groupby(['Clusters'])[c].transform('min')


In [344]:
new_df = pd.merge(df, inner_join_df, on="id", how="left")

#coalesce the clustered values, then the non-clustered ones
for c in fields_list:
    if c+'_cat' in quasi_ids_cat:
        new_df[c] = new_df[c + '_cat_y'].combine_first(new_df[c + '_cat_x'])
    else:
        new_df[c] = new_df[c + '_y'].combine_first(new_df[c + '_x'])

#clean up the columns
reduced_df=new_df[fields_list].copy()
reduced_df

#now we need to re-map the qids 


,Hectare,Date,Age,Primary Fur Color,Highlight Fur Color,Location,Specific Location,Running,Chasing,Climbing,Eating,Foraging,Other Activities,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions
0,3.0,10142018.0,1.0,2.0,4.0,1.0,NaN,False,False,False,False,False,NaN,False,False,False,False,False,False,False,False,NaN
1,0.0,10192018.0,-1.0,-1.0,-1.0,-1.0,NaN,False,False,False,False,False,NaN,False,False,False,False,False,False,False,False,NaN
2,0.0,10122018.0,-1.0,-1.0,-1.0,-1.0,NaN,False,True,False,False,False,NaN,False,False,False,False,False,False,False,False,NaN
3,0.0,10172018.0,-1.0,-1.0,-1.0,-1.0,NaN,False,False,False,True,True,NaN,False,False,False,False,False,False,False,True,NaN
4,1.0,10172018.0,1.0,2.0,4.0,0.0,on tree stump,False,False,False,False,True,NaN,False,False,False,False,False,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3018,3.0,10072018.0,1.0,2.0,-1.0,1.0,NaN,False,False,False,True,True,NaN,False,False,False,False,False,False,False,True,NaN
3019,1.0,10132018.0,1.0,2.0,9.0,1.0,NaN,False,False,False,False,True,NaN,False,False,False,False,False,False,True,False,NaN
3020,0.0,10122018.0,-1.0,-1.0,-1.0,-1.0,NaN,False,False,False,True,True,NaN,False,False,False,False,False,False,True,False,NaN
3021,0.0,10102018.0,-1.0,-1.0,-1.0,-1.0,NaN,False,False,False,True,False,NaN,False,False,False,False,False,False,True,False,NaN


In [348]:
# df_cat_mapping[quasi_ids_cat]
# quasi_ids_cat

# create a mapping dictionary to get the original strings back and maybe not get penalized
# on usefulness

d={}
for c in quasi_ids_cat:
  d[c] = dict(df_cat_mapping[[c,c.replace("_cat", "")]].drop_duplicates().to_dict('split')['data'])
  reduced_df[c.replace("_cat", "")] = reduced_df[c.replace("_cat", "")].map(d[c])





In [356]:
reduced_df


,Hectare,Date,Age,Primary Fur Color,Highlight Fur Color,Location,Specific Location,Running,Chasing,Climbing,Eating,Foraging,Other Activities,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions
0,3,10142018.0,Adult,Gray,Cinna,Ground Plane,NaN,False,False,False,False,False,NaN,False,False,False,False,False,False,False,False,NaN
1,0,10192018.0,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,NaN,False,False,False,False,False,False,False,False,NaN
2,0,10122018.0,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False,NaN,False,False,False,False,False,False,False,False,NaN
3,0,10172018.0,NaN,NaN,NaN,NaN,NaN,False,False,False,True,True,NaN,False,False,False,False,False,False,False,True,NaN
4,1,10172018.0,Adult,Gray,Cinna,Above Ground,on tree stump,False,False,False,False,True,NaN,False,False,False,False,False,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3018,3,10072018.0,Adult,Gray,NaN,Ground Plane,NaN,False,False,False,True,True,NaN,False,False,False,False,False,False,False,True,NaN
3019,1,10132018.0,Adult,Gray,Wh,Ground Plane,NaN,False,False,False,False,True,NaN,False,False,False,False,False,False,True,False,NaN
3020,0,10122018.0,NaN,NaN,NaN,NaN,NaN,False,False,False,True,True,NaN,False,False,False,False,False,False,True,False,NaN
3021,0,10102018.0,NaN,NaN,NaN,NaN,NaN,False,False,False,True,False,NaN,False,False,False,False,False,False,True,False,NaN
